# This is for:
**pretesting ka distribution in new version to avoid big problems.**

In [ ]:
# -*- coding=utf-8 -*-
# __author = 'bonnieting'__

# basic:
import datetime as dt
import os, sys
import pandas as pd
import numpy as np
import plotly as ply
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import statistics as stat
import warnings
warnings.filterwarnings('ignore')
import colorlover as cl
import copy

# time:
from dateutil.relativedelta import relativedelta
from datalab_lib.common import common_func, data_util, const, common_plot, common_htmlTable
import re
import calendar
from dateutil.parser import parse

# visual:
from plotly.offline import init_notebook_mode
import plotly.offline as py
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.figure_factory as ff
from IPython.display import Markdown, display, HTML
ply.offline.init_notebook_mode(connected=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pre_defined:
today = dt.datetime.today()
current_month = today.strftime('%Y-%m')
start_date = (today.date() - relativedelta(days=7)).strftime('%Y-%m-%d')

day = calendar.monthrange(today.year,today.month)[1]
end_date = dt.datetime(today.year ,today.month,day).strftime('%Y-%m-%d')

# get data from db
- _data_util.db_source is a method i wrote myself in the common folder to facilitate querying from databse_

In [ ]:
# query tran_volume and balance_snap in last 30 days of a fixed day in period
@data_util.db_source(db='retailbi')
def data_ka1(period):
    sql = """
    select '{period}' as stat_date,
            cid,'last_30d_trade_val' as tag,
            sum(transaction_volume) amt
    from retail_db.ba_succ_transaction
    where order_end_dt>='{period}' - interval '30' day and order_end_dt<'{period}'
          and record_type in ('BUY') and worth_coin='CNY'
    group by 1,2,3 
    
    union all 
    
    select m1.period as stat_date,cid,'total_auc' as tag,
           sum(balance_value * ifnull(price,1)) balance_val
    from retail_db.ba_user_balance_monthly_snap m1
    left join retail_db.dim_exchange_rate m2 
         on m1.balance_code=m2.digital_code 
            and m2.legal_code='CNY' 
            and m2.settle_date=m1.period 
    where period = '{period}' 
          and m1.balance_value>0 
          and rna_area != 'CHINA' 
          and m1.balance_code != 'CNY'
    group by 1,2,3
    """.format(period = period)
    return sql


@data_util.db_source(db='retailbi')
def data_ka2(update=False):
    sql = """    
    select m2.cid,level as note,expired_at
    from retail.retail_customer m1 
    join retail.retail_ka_info m2 on m1.id=m2.cid 
    where is_robot=0 and expired_at>current_date
    """
    return sql

In [ ]:
# query user_val of each period on the fixed date
@data_util.db_source(db='retail_db')
def data_trade1(date):
    sql = """    
    select
    	period,
    	m1.cid,
    	round(sum(balance_value * ifnull(price,1)),2) as balance_val,
    	m3.trade_val,
    	m3.order_cnt
    from retail.ba_user_balance m1
    left join retail.exchange_rate m2 on m1.good_code=m2.product_code and m2.legal_code='CNY' and m2.settle_date=current_date
    left join (
    	select cid,sum(transaction_volume) as trade_val,count(distinct order_id) as order_cnt
    	from retail.ba_succ_transaction force index(idx_record_type_cost_coin_order_end_dt)
    	where rna_area='CHINA' and order_end_dt<'{date}' and record_type in ('SELL','BUY')
    	group by 1
    	having trade_val>0
    )m3 on m1.cid=m3.cid
    where period='{data}'
          and rna_area='CHINA'
    group by 1,2
    order by 3 desc;
    """.format(date=date)
    return sql

# predefine function
- _there is discription of each function before one_

In [ ]:
# grade every ka with last_30d_trade_val and total_auc
# last_30d_trade_val(in CNY)
# total_auc = available + frozen + finance (in CNY)
def sample(date,i):
    df1 = data_ka1(date)
    df11 = df1.pivot_table(index=['cid'],columns='tag',values='amt',fill_value=0).reset_index()
    df11['stat_date'] = config['period'].unique()[i]
    df11['score'] = df11['last_30d_trade_val'] + df11['total_auc'] * 30
    df11['cid'] = df11['cid'].astype('str')
    
    return df11

In [ ]:
# distinct ka with note by score above
def calc_ka_score(df):
    conditions = [
        df['score'] >= 5000000, # ka3 PREMIUM
        (df['score'] >= 300000) & (df['score'] < 5000000), # ka2 REGULAR
        (df['score'] >= 30000) & (df['score'] < 300000), # ka1 BASIC
    ]

    values = ['ka3', 'ka2', 'ka1']

    df['note'] = np.select(conditions, values, default='ka0') # common users

    df = df.pivot_table(index='note',values='cid',aggfunc=lambda x: len(x.unique())).reset_index()
    df['pct'] = df['cid'] / df['cid'].sum()
    df['ka_pct'] = df['cid'] / df[df['note'] != 'ka0']['cid'].sum()
    
    return df

In [ ]:
#see how times ka_rank changed
def ka_rank_change(df):
    conditions = [
        df['note_before'] > df['note_after'], # downgrade
        df['note_before'] < df['note_after'], # upgrade
    ]

    values = ['downgrade', 'upgrade']

    df['change_note'] = np.select(conditions, values, default='no_change')

    df = df.pivot_table(index=['change_note'],
                        values='cid',
                        aggfunc=lambda x: len(x.unique())
                       ).reset_index()
    df['pct'] = df['cid'] / df['cid'].sum()
    
    return df

In [ ]:
#output the boxplot of each ka segment to see distribution of balance
def output_boxplot(df, downbound, upbound):
        df=df[downbound:upbound][:]
        df.head()
        df[['balance_val']].describe().style.format('{:.2f}')
        color = dict(boxes='DarkRed', whiskers='DarkGreen',medians='Red', caps='Red')
        pic = df[['balance_val']].plot(kind='box',title='balance Value Distribution of {}'.format(str('%.2f'%i)+'%'),color=color,sym='r+',figsize=(20,2),vert=False)
        i += 1
    return pic

# ka count dustribution
- _in new hierarchy rules_
- _**pick ten timenodes as samples to see if the ka distribution in new version in each duration meets our expectation**_
- _ka_all_cnt == 40,000_

In [ ]:
config = pd.DataFrame({'period': ['2023-07-31','2023-07-24','2023-07-17','2023-07-10','2023-07-03',
                                  '2023-06-26','2023-06-19','2023-06-12','2023-06-05','2023-05-29']}, 
                      columns = ['period']) 
dfs = []
for i in range(len(config)):
    df = sample(config['period'].unique()[i],config.index[i])
    dfs.append(df)

In [ ]:
for df in dfs:
    
    df_result = calc_ka_score(df)
    date = df['stat_date'].unique()
    ka_result = df_result.sort_values('cid',ascending=False)
    
    print(date,'\n','ka_group_pct','\n',ka_result,'\n\n')

# ka balance funnel
- _the higher level a customer has, the more balance he has, following the Pareto Principle(80/20 Rule)_

In [ ]:
df = data_trade1(today)
df.head()
df[['balance_val']].describe().style.format('{:.2f}')
TTL_user=df[['cid']].count()
TTL_user

In [ ]:
bound_list = [0, 0.015, 0.1, 0.3, 0.6, 0.95, 1]
for i in bound_list:
    downbound = user_cnt*i, upbound = user_cnt*(i+1)
    plot = output_boxplot(df, downbound, upbound)
    plot.show()

In [ ]:
label=['1.5%''10%','30%','60%','95%','100%']
value=[9067,63463,54397,36264,15413,2719]

trace = go.Funnel(
    y = label,
    x = value,
    textinfo = "value + percent initial",
    marker=dict(color=["deepskyblue", "lightsalmon", "tan", "teal", "silver", "yellow"]),
    connector = {"line": {"color": "royalblue", "dash": "solid", "width": 3}}
)
    
data =[trace]

fig = go.Figure(data)
fig.show()

df11=pd.DataFrame({'label':label,'value':value,'pct':(value/TTL_user)})
df11.add({})
df11.head()

# ka reshuffle times
- _upgrade or downgrade times to see if the hirarchy is stable enough_

In [ ]:
for i in range(1,len(dfs)):

    df_left = dfs[i-1].join(dfs[i].set_index('cid'),on='cid',lsuffix='_after',rsuffix='_before',how='left')
    df_left['note_before'] = df_left['note_before'].fillna('ka0')


    df_right = dfs[i-1].join(dfs[i].set_index('cid'),on='cid',lsuffix='_after',rsuffix='_before',how='right')
    df_right['note_after'] = df_right['note_after'].fillna('ka0')

    df_change = df_left.append(df_right).drop_duplicates()

    df_change = ka_rank_change(df_change)

    date = dfs[i-1]['stat_date'].unique()

    print(date,'\n','ka_change_pct','\n',df_change,'\n\n')

# new _VS_ old hierarchy
- _**predict from data if there will be evident problems after releasing a new version ka rules**_
- _compare in reshuffle times_
- _compare in count distribution_
- _compare in balance distribution_

In [ ]:
new = dfs[0][['cid','score','note']]

current = data_ka2()
current = current.replace({'BASICS':'ka1','PREMIUM':'ka3','REGULAR':'ka2'})
current['cid'] = current['cid'].astype('str')


df_left = new.join(current.set_index('cid'),on='cid',lsuffix='_after',rsuffix='_before',how='left')
df_left['note_before'] = df_left['note_before'].fillna('ka0')


df_right = new.join(current.set_index('cid'),on='cid',lsuffix='_after',rsuffix='_before',how='right')
df_right['note_after'] = df_right['note_after'].fillna('ka0')

df_change = df_left.append(df_right).drop_duplicates()

ka_rank_change(df_change)

In [ ]:
df_change.pivot_table(index='note_after',
                      columns='note_before',
                      values='cid',
                      aggfunc=lambda x: len(x.unique()),
                      margins=True)

In [ ]:
ka_20230601 = current.copy()

In [ ]:
ka_20230602 = data_ka2()

ka_20230602 = ka_20230602.replace({'BASICS':'ka1','PREMIUM':'ka3','REGULAR':'ka2'})
ka_20230602['cid'] = ka_20230602['cid'].astype('str')


df_left = ka_20230602.join(ka_20230601.set_index('cid'),on='cid',lsuffix='_after',rsuffix='_before',how='left')
df_left['note_before'] = df_left['note_before'].fillna('ka0')


df_right = ka_20230602.join(ka_20230601.set_index('cid'),on='cid',lsuffix='_after',rsuffix='_before',how='right')
df_right['note_after'] = df_right['note_after'].fillna('ka0')

df_change = df_left.append(df_right).drop_duplicates()

ka_rank_change(df_change)

In [ ]:
df_change.pivot_table(index='note_after',
                      columns='note_before',
                      values='cid',
                      aggfunc=lambda x: len(x.unique()),
                      margins=True)

In [ ]:
ka_20230601.pivot_table(index='note',
                      values='cid',
                      aggfunc=lambda x: len(x.unique()),
                      margins=True).T

In [ ]:
ka_20230602.pivot_table(index='note',
                      values='cid',
                      aggfunc=lambda x: len(x.unique()),
                      margins=True).T

In [ ]:
common_func.hide_code()